# Webscraping & Data Transformation of Book Results 📚🔎

## Step 1: Installing and importing relevant python libraries 🔎

#### Start by installing the relevant Python library. For this demonstration, I am using "__beautifulsoup4__" and "__mysql-connector-python__".

In [ ]:
pip install requests beautifulsoup4


In [ ]:
pip install mysql-connector-python


In [ ]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
import time
import re

*I start by installing BeautifulSoup4 a Python library perfect for scraping data straight off webpages-gives me all the tools to yank tags, text, links etc.*

## Step 2: Database Connection Setup

In [ ]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Lauraandlunaforever1993",
    database="bookstoscrape"
)

cursor = conn.cursor()

## Step 3: Defining The URL's Required

In [ ]:
url = 'https://books.toscrape.com/catalogue/page-{}.html'

BASE_URL = 'https://books.toscrape.com/catalogue/'

## Step 4: Get Page Content

#### I create a "get_page_content" variable which fetches the HTML and parses through to BeautifulSoup

In [ ]:
def get_page_content(page_url):
    response = requests.get(page_url)
    return BeautifulSoup(response.content, 'html.parser') 


## Step 5: Retreaving The Book Details __("Instock Availability" & "Genre")__.

*It looks for the p class=instock availability tag on the page. Then it searches the text for the number available. If it then finds the number available, it then converts the data type into an integer. It then looks inside the breadcrumb navigation link class=breadcrumb It takes the genre type and extracts the text.*

*With the return values, if it isn't able to identify a number from how many are available it will then return "0" as the default. If the Genre isn't extracted it will return none if it's not found.*

In [ ]:
def get_book_details(detail_url):
    detail_soup = get_page_content(detail_url)

    availability_tag = detail_soup.find('p', class_='instock availability')
    number_available = 0
    if availability_tag:
        match = re.search(r'\((\d+) available\)', availability_tag.get_text())
        if match:
            number_available = int(match.group(1))

    genre = None
    breadcrumb = detail_soup.find('ul', class_='breadcrumb')
    if breadcrumb:
        items = breadcrumb.find_all('li')
        if len(items) >= 3:
            genre_tag = items[-2].find('a')
            if genre_tag:
                genre = genre_tag.get_text(strip=True)

    return number_available, genre


## Step 6: Getting The Book Details __("Title, "Price", "Rating", "Availability")__.

*This function "scrape_book_details" extracts all the important information about a book from the HTML elements. It starts by pulling the title, cost, star rating. The "rating_mapping" is a dictionary which assigns the text to it's corresponding numerical values. The Rating is then assigned ot the "rating_mapping" with "(Rating_text, 0.0)" where any text without any values to be converted to 0. The "re.seach" function is used to find the numeric value in the stock availability test. Using the regex pattern it will extract the number from that. Then the interger is set for the data type under "Number_Available" and if match extracts the number, and the else 0 if it's unable to convert to an interger.*  

*Next a link to the book’s detail page, uses get_book_details to fetch the genre from that pages breadcrumb, and finally returns everything as a dictionary containing the title, cost, rating, stock status, number available, and genre.

In [ ]:
def scrape_book_details(book_container):
    Title = book_container.find('h3').find('a')['title']
    Cost = book_container.find('p', class_='price_color').text.strip('£').replace('Â', '')
    Rating_text = book_container.find('p', class_='star-rating')['class'][1]
    Stock_Availability = book_container.find('p', class_='instock availability').text.strip()
    
    
    match = re.search(r'\((\d+) available\)', Stock_Availability)
    Number_Available = int(match.group(1)) if match else 0

    rating_mapping = {
        "One": 1.0,
        "Two": 2.0,
        "Three": 3.0,
        "Four": 4.0,
        "Five": 5.0
    }
    Rating = rating_mapping.get(Rating_text, 0.0)

    detail_href = book_container.find('h3').find('a')['href']
    detail_url = BASE_URL + detail_href.replace('../../', '')

    number_available, genre = get_book_details(detail_url)

    return {
        "Title": Title,
        "Cost": Cost,
        "Rating": Rating,
        "Stock_Availability": Stock_Availability,
        "Number_Available": number_available,
        "Genre": genre
    }

## Step 7: Scraping Each Page.

*I create a "scrape_books_from_page" variable which parses the page_url to BeautifulSoup. The "soup.find_all" method which finds the ('article', class_='product_pod') on that page under " book_containers".*   

*This is then placed into a list and a for loop is applied to the container in "book_containers" and is then added to the "scrape_book_details" container where the other data sets are stored.*

In [ ]:
def scrape_books_from_page(page_url):
    soup = get_page_content(page_url)
    book_containers = soup.find_all('article', class_='product_pod')
    books = []
    for container in book_containers:
        books.append(scrape_book_details(container))
    return books


## Step 8: Test The Result Of The First Page.

*A for loop is created to scan through "books_on_first_page" to scrape through the first page and pulls the books details of the title, cost, rating, stock availability, number available, and genre. It then prints out those details or breaks the loop with the "is not None" and handles it to the else print, to state "Warning: Book details could not be scraped".*

In [ ]:
page_url = url.format(1)
books_on_first_page = scrape_books_from_page(page_url)

for book in books_on_first_page:
    if book is not None:
        print(f"Title: {book['Title']}, Cost: £{book['Cost']}, Rating: {book['Rating']} "
              f"Stock_Availability: {book['Stock_Availability']} "
              f"Number_Available: {book['Number_Available']} Genre: {book['Genre']}")
    else:
        print("Warning: Book details could not be scraped.")


## Step 9: Scraping All Pages.

*A while loop is created to itterate through each page. The page number is first defined as 1 "page_number = 1" and is called to the url to be formatted.*  

*The "scrape_books_from_page" then contains the page number with the list of the "scrape_book_details" and brings together with "full_book_and_containers_on_page".*

*After that the if not statement is applied and the break to stop the loop if there a no longer any pages to go through. This is all stored under a list under "all_books" = [] under the "all_books.extend" method.*

*To find the results, I then print and call the page number and the number of books within each given page, the "page_number += 1" allows for page-by page increase to loop through after each result is printed until the if not method.*

*A timer is applied for two seconds to allow for a delay for each page before continuing the scrape to prevent any overloading with too many requests.*

In [ ]:
all_books = []
page_number = 1

while True:
    page_url = url.format(page_number)
    
    full_book_and_containers_on_page = scrape_books_from_page(page_url)

    if not full_book_and_containers_on_page:
        break
    
    all_books.extend(full_book_and_containers_on_page)

    print(f"Page {page_number}: Found {len(full_book_and_containers_on_page)} books")
    
    page_number += 1
    time.sleep(2)


## Step 10: Saving To The MYSQL Database.

#### Then a for loop is used in "all_books" to save to the database the "Title", "Cost", "Rating", "Stock_Availability", "Genre" and "Number_Available".

*This code stores all the scraped books into the database. The function "savetoDB" inserts each book’s details title, cost, rating, stock availability, genre, and number available into the bookdata table. To ensure there were no duplicate values the "ON DUPLICATE KEY UPDATE" is used to prevent this from happening. The for loop calls every book in "all_books" and saves any missing genres as "None". The script carries out the changes to the database and closes the cursor and the connection.*

In [ ]:
def savetoDB(title, cost, rating, stock, genre, number):
    cursor.execute("""
        INSERT INTO bookdata
        (Title, Cost, Rating, Stock_Availability, Genre, Number_Available)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            Cost = VALUES(Cost),
            Rating = VALUES(Rating),
            Stock_Availability = VALUES(Stock_Availability),
            Genre = VALUES(Genre),
            Number_Available = VALUES(Number_Available)
    """, (title, cost, rating, stock, genre, number))

for book in all_books:
    if book is not None:
        savetoDB(
            book["Title"],
            book["Cost"],
            book["Rating"],
            book["Stock_Availability"],
            book["Genre"] if book["Genre"] else None,
            book["Number_Available"]
        )

conn.commit()
cursor.close()
conn.close()



__Here is the mark down file for the MYSQL table results:__    
https://github.com/Drook93/Webscrape-Books/blob/master/SQL_Results_Books_Scraped.md
    

__Here is the PowerPoint presentation for the data analysis project:__   

https://github.com/Drook93/Webscrape-Books/blob/master/Website%20Scraping%20Project.pdf


